In [109]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [110]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [111]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [112]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=0, discrete_branches=(1,)))


In [113]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [114]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 0, Discrete: (1,)


In [115]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [116]:
print(decision_steps)
print(list(decision_steps))

[0]


In [117]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [118]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[[0]]


In [119]:
env.set_actions(behavior_name, action)

In [120]:
for episode in range(30):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        # action = spec.create_random_action(len(decision_steps))
        action = spec.action_spec.empty_action(len(decision_steps))
        action.add_discrete(np.array([[0]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

[array([[ 4.1899934, -1.1772   , 19.556175 , -1.       ,  8.144831 ,
         8.086967 , 10.940731 , 11.611248 ]], dtype=float32)]
[array([[ 4.1899934, -2.1582   , 19.556175 , -1.       ,  8.063625 ,
         8.122432 , 10.782094 , 11.646557 ]], dtype=float32)]
[array([[ 4.1899934, -3.1391995, 19.556175 , -1.       ,  7.929895 ,
         8.149343 , -1.       , 11.680294 ]], dtype=float32)]
[array([[ 4.1236367 ,  0.09439678, 19.325298  , -1.        ,  7.8582397 ,
         8.169605  , 11.635565  , 12.389982  ]], dtype=float32)]
[array([[ 4.1508603 ,  0.07976671, 19.347403  , -1.        ,  7.7400937 ,
         8.195798  , 10.816745  , 12.436833  ]], dtype=float32)]
[array([[ 4.1711626 ,  0.11212192, 19.312712  , -1.        ,  7.6207094 ,
         8.22392   , 10.525336  , 12.496689  ]], dtype=float32)]
[array([[ 4.1989117,  0.2063475, 19.34544  , -1.       ,  7.4588437,
         8.254282 ,  9.853192 , 12.909299 ]], dtype=float32)]
[array([[ 4.329537  ,  0.22007534, 19.337648  , -1.        

In [121]:
env.close()